In [117]:
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

In [118]:
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [119]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [120]:
tf.compat.v1.disable_eager_execution()

In [121]:
import matplotlib.pyplot as plt
import numpy as np
import os

In [136]:
#initializing the CNN
classifier = Sequential()

In [137]:
classifier.add(Conv2D(32, (3, 3), input_shape = (128, 128, 3), activation = 'relu'))

In [138]:
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (3, 3)))

In [139]:
classifier.add(Flatten())

In [140]:
#hidden layer
classifier.add(Dense(units = 128, activation = 'relu'))
#output layer
classifier.add(Dense(units = 5, activation = 'sigmoid'))

In [141]:
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)


In [142]:
training_set = train_datagen.flow_from_directory('C:/Users/student/Desktop/dataset/Train', # relative path from working directoy
                                                 target_size = (128, 128),
                                                 batch_size = 10, class_mode = 'categorical')

Found 53 images belonging to 5 classes.


In [143]:
valid_set = test_datagen.flow_from_directory('C:/Users/student/Desktop/dataset/val', # relative path from working directoy
                                             target_size = (128, 128), 
                                        batch_size = 6, class_mode = 'categorical')

Found 28 images belonging to 5 classes.


In [144]:
labels = (training_set.class_indices)
print(labels)

{'berry_blotch': 0, 'berry_disease': 1, 'die_back': 2, 'healthy': 3, 'unknown': 4}


In [145]:
classifier.fit(training_set,
                         steps_per_epoch = 20,
                         epochs = 24,
                         validation_data=valid_set
                         )

Epoch 1/24
20/20 [==============================] - 49s 2s/step - batch: 9.5000 - size: 8.6000 - loss: 2.3774 - accuracy: 0.2442 - val_loss: 1.5424 - val_accuracy: 0.2857
Epoch 2/24
20/20 [==============================] - 45s 2s/step - batch: 9.5000 - size: 8.9500 - loss: 1.3252 - accuracy: 0.3911 - val_loss: 1.4841 - val_accuracy: 0.2857
Epoch 3/24
20/20 [==============================] - 43s 2s/step - batch: 9.5000 - size: 8.9500 - loss: 0.9638 - accuracy: 0.6816 - val_loss: 1.4049 - val_accuracy: 0.4286
Epoch 4/24
20/20 [==============================] - 42s 2s/step - batch: 9.5000 - size: 8.9500 - loss: 0.6631 - accuracy: 0.7933 - val_loss: 2.8246 - val_accuracy: 0.3929
Epoch 5/24
20/20 [==============================] - 41s 2s/step - batch: 9.5000 - size: 8.6000 - loss: 0.5969 - accuracy: 0.8023 - val_loss: 2.1107 - val_accuracy: 0.6071
Epoch 6/24
20/20 [==============================] - 48s 2s/step - batch: 9.5000 - size: 8.9500 - loss: 0.3274 - accuracy: 0.8827 - val_loss: 2.30

In [146]:
classifier_json=classifier.to_json()
with open("model5.json", "w") as json_file:
    json_file.write(classifier_json)

In [147]:
# serialize weights to HDF5
classifier.save_weights("my_model_weights.h5")
classifier.save("model4.h5")
print("Saved model to disk")

Saved model to disk


In [176]:
import tensorflow as tf
model=tf.keras.models.load_model("model4.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.experimental_new_converter = True
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model) 

INFO:tensorflow:Assets written to: C:\Users\student\AppData\Local\Temp\tmp3h5bou5f\assets


INFO:tensorflow:Assets written to: C:\Users\student\AppData\Local\Temp\tmp3h5bou5f\assets


27588508

In [181]:
import datetime
import tensorboard
%load_ext tensorboard
%tensorboard --logdir logs/fit

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 10040), started 0:00:45 ago. (Use '!kill 10040' to kill it.)